In [1]:
import json
import pandas as pd
import random

In [2]:
with open("/u/ggabra/Search Project/yelp_academic_dataset_review.json") as f:
    data = pd.DataFrame(json.loads(line) for line in f)

In [3]:
data.head(1)

,business_id,date,review_id,stars,text,type,user_id,votes
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{'cool': 0, 'funny': 0, 'useful': 0}"


In [4]:
data['text'][0]

'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.'

In [5]:
rows = random.sample(range(0,len(data)-1),268500)

In [6]:
testData = data.iloc[rows,[3,4]].copy()  

In [7]:
testData.head(1)

,stars,text
2633897,2,"Over...wait for it...rated.\n\nOkay, so the Lo..."


In [8]:
len(testData)

268500

In [9]:
testData=testData.reset_index(drop=True)

In [10]:
trainData = data.drop(rows)

In [11]:
len(trainData)

2416566

In [12]:
trainData=trainData.reset_index(drop=True)

In [13]:
trainData.drop(trainData.columns[[0,1,2,5,6,7]], axis=1, inplace=True)

In [14]:
f = open("testing.json", "w")
for row in testData.iterrows():
    row[1].to_json(f)
    f.write("\n")
f.close();    

In [17]:
f = open("training.json", "w")
for row in trainData.iterrows():
    row[1].to_json(f)
    f.write("\n")
f.close();

In [14]:
train_corpus = []
train_label = []
count=0
for i,row in trainData.iterrows():
    if len(train_corpus) > 10000:
        break
    train_corpus.append(row['text'])
    train_label.append(row['stars'])
#def end_of_loop():
#    raise StopIteration
#try:    
#    train_corpus = [row['text'] if len(train_corpus) < 100 else end_of_loop() for i,row in trainData.iterrows()]
#except StopIteration:
#    pass


In [15]:
train_corpus[3]

'PROS: Italian hoagie was delicious.  Friendly counter employee. The restaurant was clean and neat. \n\nCONS: The pizza was not good.  Pre-formed crust, NOT fresh dough.  The price of the failure of a pizza WAS NOT CHEAP EITHER.  \n\nI guess the name says it all.  Get the hoagie, pass on the pizza.'

In [20]:


#vectorizer = CountVectorizer(stop_words="english",token_pattern='[a-zA-Z]*',max_features=100000)
#x_train = vectorizer.fit_transform(train_corpus)
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(x_train)

#feature_names = vectorizer.get_feature_names()
#clf = MultinomialNB().fit(X_train_tfidf, train_label)
#print(feature_names)




In [16]:
test_corpus = []
test_label = []
count=0
for i,row in testData.iterrows():
    if len(test_corpus) > 1000:
        break
    test_corpus.append(row['text'])
    test_label.append(row['stars'])

#vectorizer_test = CountVectorizer(stop_words="english",max_features=100000)
#x_test = vectorizer.transform(test_corpus)
#X_test_tfidf = tfidf_transformer.transform(x_test)


In [19]:
#predicted_label = clf.predict(X_test_tfidf)
#numpy.mean(predicted_label == test_label)

0.44155844155844154

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy
text_clf = Pipeline([('vect', CountVectorizer(stop_words="english",token_pattern='[a-zA-Z]*',ngram_range=[1,2])),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())              #change classifier here
 ])
#SGDClassifier(loss='hinge', penalty='l2',
                                            #alpha=1e-3, n_iter=5, random_state=42)
_ = text_clf.fit(train_corpus, train_label)

In [49]:
predicted_label = text_clf.predict(test_corpus)
numpy.mean(predicted_label == test_label)

0.42857142857142855

In [43]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(test_label, predicted_label))

1.269011272759076

In [46]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
text_clf = Pipeline([('vect', CountVectorizer(stop_words="english",token_pattern='[a-zA-Z]*',ngram_range=[1,2])),
                      ('tfidf', TfidfTransformer()),
                      ('clf', DecisionTreeClassifier())              #change classifier here
 ])
#LogisticRegression(solver='newton-cg', multi_class='multinomial').fit(train_x,truth)
_ = text_clf.fit(train_corpus, train_label)

In [39]:
predicted_label = text_clf.predict(test_corpus)

In [40]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(test_label, predicted_label))

1.274117978594064

In [69]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [22]:
from nltk.corpus import sentiwordnet as swn

In [28]:
breakdown = swn.senti_synset('breakdown.n.01')
print(breakdown)

<dislocation.n.02: PosScore=0.0 NegScore=0.0>


In [ ]:
import nltk
text = nltk.word_tokenize("n't")
tagWordList = nltk.pos_tag(text)